<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# FXCM Algorithmic Trading Initiative

## Python & Historical Tick Data

**Dr. Yves J. Hilpisch**


The Python Quants GmbH

**FXCM Webinar, 24. October 2017**

<img src="http://hilpisch.com/images/finaince_visual_low.png" width=300px align=left>

## Risk Disclaimer

<font size="-1">
Trading forex/CFDs on margin carries a high level of risk and may not be suitable for all investors as you could sustain losses in excess of deposits. Leverage can work against you. Due to the certain restrictions imposed by the local law and regulation, German resident retail client(s) could sustain a total loss of deposited funds but are not subject to subsequent payment obligations beyond the deposited funds. Be aware and fully understand all risks associated with the market and trading. Prior to trading any products, carefully consider your financial situation and experience level. Any opinions, news, research, analyses, prices, or other information is provided as general market commentary, and does not constitute investment advice. FXCM & TPQ will not accept liability for any loss or damage, including without limitation to, any loss of profit, which may arise directly or indirectly from use of or reliance on such information.
</font>

## First Steps with pandas

In [ ]:
import pandas as pd

### Reading Data from a CSV File

In [ ]:
# data file provided by FXCM Financial Capital Markets Ltd.
file_location = 'http://hilpisch.com/eurusd.csv'

In [ ]:
%time data = pd.read_csv(file_location, index_col=0, parse_dates=True)

In [ ]:
data.info()

In [ ]:
data.head()

### Visualizing Data

In [ ]:
from pylab import plt
plt.style.use('seaborn')
%matplotlib inline

In [ ]:
data['CloseAsk'].plot(figsize=(10, 6), lw=0.8);

### Adding Statistics

In [ ]:
data = data.loc['2014-1-1':]

In [ ]:
data['CloseMid'] = data[['CloseAsk', 'CloseBid']].mean(axis=1)

In [ ]:
data['SMA1'] = data['CloseMid'].rolling(10).mean()
data['SMA2'] = data['CloseMid'].rolling(60).mean()

In [ ]:
data.dropna(inplace=True)

In [ ]:
data[['CloseMid', 'SMA1', 'SMA2']].plot(figsize=(10, 6), lw=0.8);

### Deriving Positions

In [ ]:
import numpy as np

In [ ]:
data['Positions'] = np.where(data['SMA1'] > data['SMA2'], 1, -1)

In [ ]:
data[['CloseMid', 'SMA1', 'SMA2', 'Positions']].plot(figsize=(10, 6),
                                        secondary_y='Positions', lw=0.8);

### Backtesting the Strategy

In [ ]:
data['Returns'] = np.log(data['CloseMid'] / data['CloseMid'].shift(1))

In [ ]:
data['Strategy'] = data['Positions'].shift(1) * data['Returns']

In [ ]:
data[['Returns', 'Strategy']].dropna().cumsum().apply(np.exp).plot(
                                figsize=(10, 6), lw=0.8);

## Reading FXCM Tick Data

### The Tick Reader Class

In [ ]:
from fxcm_tick_reader import fxcm_tick_reader

### Available Symbols

In [ ]:
fxcm_tick_reader.get_available_symbols()

### Retrieving Tick Data

In [ ]:
import pandas as pd
import datetime as dt

In [ ]:
start = dt.datetime(2017, 10, 19)
stop = dt.datetime(2017, 10, 20)

In [ ]:
%time td = fxcm_tick_reader('EURUSD', start, stop)

In [ ]:
type(td)

In [ ]:
td.get_raw_data().info()

In [ ]:
td.get_raw_data().tail(10)

## Working with the Tick Data

In [ ]:
%time td.get_data().info()

In [ ]:
%time td.get_data().info()

In [ ]:
%%time
data = td.get_data(start='2017-10-20 08:00:00', end='2017-10-20 16:00:00')
data.info()

In [ ]:
from pylab import plt
plt.style.use('seaborn')
%matplotlib inline

In [ ]:
data['Bid'].plot(figsize=(10, 6), lw=0.8);

## Technical Indicators

### Creating a Sub-Set of the Data

In [ ]:
data.info()

In [ ]:
df = data.resample('1min', label='right').last().ffill()

In [ ]:
df['Mid'] = df.mean(axis=1)

In [ ]:
df.info()

### Creating the Main Object

In [ ]:
import fxcm_ti

In [ ]:
ta = fxcm_ti.technical_indicators(df)

### Simple Moving Average (SMA)

In [ ]:
ta.sma('Mid', 20).dropna().head()

In [ ]:
ta.sma('Mid', 20).plot(figsize=(10, 6));

In [ ]:
sma1_name = ta.add_sma('Mid', 30)
sma2_name = ta.add_sma('Mid', 60)

In [ ]:
ta.get_data()[['Mid', sma1_name, sma2_name]].plot(figsize=(10, 6), lw=0.9);

## Bollinger Band

In [ ]:
bollu = ta.add_bollinger_lower('Mid', 50)
bolld = ta.add_bollinger_upper('Mid', 50)

In [ ]:
ta.get_data()[['Mid', bollu, bolld]].plot(figsize=(10, 6), lw=0.8,
                                          style=['-', 'm--', 'm--']);

### Relative Strength Index (RSI)

In [ ]:
rsi_name = ta.add_rsi('Mid', 14)
rsi_name

In [ ]:
ta.get_data()[['Mid', rsi_name]].plot(figsize=(10, 6),
                                lw=0.8, subplots=True);

### Combining Indicators

In [ ]:
ta.get_data().info()

In [ ]:
f, (ax1, ax2) = plt.subplots(2, sharex=True, figsize=(10, 10))
ta.get_data()[['Mid', sma1_name, bollu, bolld]].plot(
                    style=['-', '-', 'm--', 'm--'], lw=0.8, ax=ax1)
ta.get_data()[rsi_name].plot(lw=0.8, ax=ax2)
ax2.axhline(30, c='r', ls='--', lw=0.5)
ax2.axhline(70, c='r', ls='--', lw=0.5);

### Creating an OHLC Data Set

In [ ]:
data.info()

In [ ]:
data['Mid'] = data.mean(axis=1)

In [ ]:
df = pd.DataFrame()

In [ ]:
freq = '1min'

In [ ]:
df['Open'] = data['Mid'].resample(freq, label='right').first()

In [ ]:
df['High'] = data['Mid'].resample(freq, label='right').max()

In [ ]:
df['Low'] = data['Mid'].resample(freq, label='right').min()

In [ ]:
df['Close'] = data['Mid'].resample(freq, label='right').last()

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.plot(figsize=(10, 6), lw=0.8);

## plotly & Cufflinks

**Requires http://plot.ly account.**

In [ ]:
import cufflinks as cf

In [ ]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

### Regular Interactive Plots

In [ ]:
iplot(df['Close'].iplot(kind='lines', color='blue', width=2,
           bestfit=True, title='EUR/USD', asFigure=True))

### Basic Quant Figures

In [ ]:
qf = cf.QuantFig(df, title='EUR/USD', legend='top',
                 name='EUR/USD', datalegend=False)

In [ ]:
iplot(qf.iplot(asFigure=True))

## Adding Studies

### Simple Moving Average (SMA)

In [ ]:
qf.add_sma([10, 30, 60], colors=['green', 'orange', 'blue'], width=1)
qf.studies['sma']['display'].update(legendgroup=True, showlegend=False)

In [ ]:
iplot(qf.iplot(asFigure=True))

### Bollinger Band

In [ ]:
qf.add_bollinger_bands(periods=20, boll_std=2, colors=['magenta', 'grey'], fill=True)
qf.data.update()

In [ ]:
iplot(qf.iplot(asFigure=True))

### Relative Strength Index (RSI)

In [ ]:
qf.add_rsi(periods=20, rsi_upper=70, rsi_lower=30, legend=False)

In [ ]:
iplot(qf.iplot(asFigure=True))

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>